In [1]:
import sqlite3
from pathlib import Path

In [58]:
conn = sqlite3.connect('../../db/ccrs.db')

cursor = conn.cursor()

In [59]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

In [60]:
table_names = [row[0] for row in cursor.fetchall()]

In [61]:
print(table_names)

['notes', 'hanzi_components', 'hanzi_info', 'radicals', 'ce_dictionary', 'example_sentences', 'cedict_sentences', 'hanzi_cedict']


In [35]:
import sqlite3
from typing import List, Tuple

def get_table_schema(table_name: str) -> List[Tuple]:
    conn = sqlite3.connect('../../db/ccrs.db')
    cursor = conn.cursor()
    query = f"PRAGMA table_info({table_name})"
    cursor.execute(query)
    result_set = cursor.fetchall()
    conn.close()
    return result_set


## Hanzi

In [15]:

query = f"PRAGMA table_info({table_name})"
result_set = conn.execute(query)
for row in result_set:
    print(row)


(0, 'hanzi', 'TEXT', 0, None, 0)
(1, 'raw_frequency', 'REAL', 0, None, 0)
(2, 'pinyin', 'TEXT', 0, None, 0)
(3, 'definition', 'TEXT', 0, None, 0)
(4, 'stroke_count', 'TEXT', 0, None, 0)
(5, 'hanzi_index', 'INTEGER', 0, None, 0)


In [143]:
class Hanzi:
    def __init__(self, hanzi: str, raw_frequency: float, pinyin: str, definition: str, stroke_count: int, hanzi_index: int):
        self.hanzi = hanzi
        self.raw_frequency = raw_frequency
        self.pinyin = pinyin
        self.definition = definition
        self.stroke_count = stroke_count
        self.hanzi_index = hanzi_index

In [144]:

from typing import Union, List

def get_hanzi_by_index(conn: sqlite3.Connection, hanzi_index: Union[int, List[int]]) -> Union[Hanzi, List[Hanzi]]:
    cursor = conn.cursor()
    if isinstance(hanzi_index, int):
        cursor.execute(f"SELECT * FROM hanzi_info WHERE hanzi_index = ?", (hanzi_index,))
        row = cursor.fetchone()
        if row is None:
            return None
        else:
            return Hanzi(hanzi=row[0], raw_frequency=row[1], pinyin=row[2], definition=row[3], stroke_count=row[4], hanzi_index=row[5])
    elif isinstance(hanzi_index, list):
        cursor.execute(f"SELECT * FROM hanzi_info WHERE hanzi_index IN ({','.join('?'*len(hanzi_index))})", hanzi_index)
        rows = cursor.fetchall()
        if len(rows) == 0:
            return None
        else:
            return [Hanzi(hanzi=row[0], raw_frequency=row[1], pinyin=row[2], definition=row[3], stroke_count=row[4], hanzi_index=row[5]) for row in rows]
    else:
        raise TypeError(f"hanzi_index must be an integer or a list of integers, not {type(hanzi_index).__name__}")


In [145]:
sample_char =  get_hanzi_by_index(conn,hanzi_index=[1005072,1003000])

In [146]:
sample_char

[<__main__.Hanzi at 0x7fcbe3694460>, <__main__.Hanzi at 0x7fcbe3655630>]

In [147]:
table_name = 'radicals'

In [148]:
rows_radicals = get_table_schema('radicals')

In [149]:
print(rows_radicals)

[(0, 'radical_number', 'INTEGER', 0, None, 0), (1, 'radical', 'TEXT', 0, None, 0), (2, 'variants', 'TEXT', 0, None, 0), (3, 'pinyin', 'TEXT', 0, None, 0), (4, 'stroke_count', 'INTEGER', 0, None, 0), (5, 'ucn', 'TEXT', 0, None, 0), (6, 'meaning', 'TEXT', 0, None, 0), (7, 'traditional', 'TEXT', 0, None, 0), (8, 'radical_index', 'INTEGER', 0, None, 0)]


In [150]:
class Radical:
    def __init__(self, radical_number: int, radical: str, variants: str, pinyin: str, stroke_count: int, ucn: str, meaning: str, traditional: str, radical_index: int):
        self.radical_number = radical_number
        self.radical = radical
        self.variants = variants
        self.pinyin = pinyin
        self.stroke_count = stroke_count
        self.ucn = ucn
        self.meaning = meaning
        self.traditional = traditional
        self.radical_index = radical_index


In [151]:

def get_radical_by_index(conn: sqlite3.Connection, radical_index: int) -> Radical:
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM radicals WHERE radical_index = ?", (radical_index,))
    row = cursor.fetchone()
    if row is None:
        return None
    else:
        return Radical(radical_number=row[0], radical=row[1], variants=row[2], pinyin=row[3], stroke_count=row[4], ucn=row[5], meaning=row[6], traditional=row[7], radical_index=row[8])


## CE DICT

In [152]:
table_idx = -1
table_names


['notes',
 'hanzi_components',
 'hanzi_info',
 'radicals',
 'ce_dictionary',
 'example_sentences',
 'cedict_sentences',
 'hanzi_cedict']

In [153]:
get_table_schema(table_names[table_idx])

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'cedict_index', 'INTEGER', 0, None, 0),
 (2, 'hanzi_index', 'INTEGER', 0, None, 0)]

In [154]:
class Word:
    def __init__(self, traditional: str, simplified: str, pinyin: str, english: str, cedict_index: int):
        self.traditional = traditional
        self.simplified = simplified
        self.pinyin = pinyin
        self.english = english
        self.cedict_index = cedict_index


In [155]:
import sqlite3

def get_cedict_by_index(conn: sqlite3.Connection, cedict_index: int) -> Word:
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM ce_dict WHERE cedict_index = ?", (cedict_index,))
    row = cursor.fetchone()
    if row is None: 
        return None
    else:
        return Word(traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3], cedict_index=row[4])
    

def search_cedict_by_simplified(conn: sqlite3.Connection, search_str: str) -> List[Word]:
    cursor = conn.cursor()
    query = f"SELECT * FROM ce_dict WHERE simplified LIKE '%' || ? || '%'"
    cursor.execute(query, (search_str,))
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        return [Word(traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3], cedict_index=row[4]) for row in rows]


def search_cedict_by_pinyin(conn: sqlite3.Connection, search_str: str) -> List[Word]:
    cursor = conn.cursor()
    query = f"SELECT * FROM ce_dict WHERE pinyin LIKE '%' || ? || '%'"
    cursor.execute(query, (search_str,))
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        return [Word(traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3], cedict_index=row[4]) for row in rows]


import sqlite3
from typing import List

def search_cedict_by_english_definition(conn: sqlite3.Connection, search_str: str) -> List[Word]:
    cursor = conn.cursor()
    query = f"SELECT * FROM ce_dict WHERE english LIKE '%' || ? || '%'"
    cursor.execute(query, (search_str,))
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        return [Word (traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3], cedict_index=row[4]) for row in rows]


## Example Sentences

In [156]:
table_names

['notes',
 'hanzi_components',
 'hanzi_info',
 'radicals',
 'ce_dictionary',
 'example_sentences',
 'cedict_sentences',
 'hanzi_cedict']

In [157]:
get_table_schema('example_sentences')

[(0, 'Characters', 'TEXT', 0, None, 0),
 (1, 'Pinyin', 'TEXT', 0, None, 0),
 (2, 'Meaning', 'TEXT', 0, None, 0),
 (3, 'HSK average', 'REAL', 0, None, 0),
 (4, 'Custom Ratio', 'REAL', 0, None, 0),
 (5, 'sentence_index', 'INTEGER', 0, None, 0)]

In [158]:
class ExampleSentence:
    def __init__(self, characters: str, pinyin: str, meaning: str, hsk_average: float, custom_ratio: float, sentence_index: int):
        self.characters = characters
        self.pinyin = pinyin
        self.meaning = meaning
        self.hsk_average = hsk_average
        self.custom_ratio = custom_ratio
        self.sentence_index = sentence_index


In [159]:
import sqlite3
from typing import List

def get_sentences_by_word_indices(conn: sqlite3.Connection, sentence_indices: List[int]) -> List[ExampleSentence]:
    cursor = conn.cursor()
    query = f"SELECT * FROM example_sentence WHERE sentence_index IN ({','.join(['?']*len(sentence_indices))})"
    cursor.execute(query, sentence_indices)
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        return [ExampleSentence(characters=row[0], pinyin=row[1], meaning=row[2], hsk_average=row[3], custom_ratio=row[4], sentence_index=row[5]) for row in rows]


Hanzi Components

In [160]:
get_table_schema('hanzi_components')

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'hanzi_index', 'INTEGER', 0, None, 0),
 (2, 'component', 'TEXT', 0, None, 0),
 (3, 'position', 'INTEGER', 0, None, 0)]

In [161]:
class HanziComponent:
    def __init__(self, index: int, hanzi_index: int, component: str, position: int):
        self.index = index
        self.hanzi_index = hanzi_index
        self.component = component
        self.position = position


In [162]:
import sqlite3
from typing import List

def get_components_by_hanzi_index(conn: sqlite3.Connection, hanzi_index: int) -> List[HanziComponent]:
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM hanzi_components WHERE hanzi_index = ?", (hanzi_index,))
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        return [HanziComponent(index=row[0], hanzi_index=row[1], component=row[2], position=row[3]) for row in rows]


In [163]:
import sqlite3
from typing import List

def get_hanzi_by_cedict_index(conn: sqlite3.Connection, cedict_index: int) -> List[Hanzi]:
    cursor = conn.cursor()
    cursor.execute(f"SELECT hanzi_index FROM hanzi_cedict WHERE cedict_index = ?", (cedict_index,))
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:
        hanzi_indices = [row[0] for row in rows]
        return [get_hanzi_by_index(conn, index) for index in hanzi_indices]


In [164]:
charlist =  get_hanzi_by_cedict_index(conn,2105610)

In [173]:
hanziTest = charlist[0]


In [166]:
import sqlite3


def query_ce_dictionary_by_hanzi_index(cursor, hanzi):
    # Define the SQL query as an f-string
    query = f"SELECT DISTINCT traditional, simplified, pinyin, english, cedict_index FROM ce_dictionary " \
            f"INNER JOIN hanzi_cedict ON ce_dictionary.cedict_index = hanzi_cedict.cedict_index " \
            f"INNER JOIN hanzi_info ON hanzi_cedict.hanzi_index = hanzi_info.hanzi_index " \
            f"WHERE hanzi_info.hanzi_index = {hanzi.hanzi_index}"

    # Execute the query
    cursor.execute(query)

    # Fetch the results
    results = cursor.fetchall()

    # Return the results
    return results


In [196]:
def query_ce_dictionary_by_hanzi_index(cursor, hanzi):

	query = """ SELECT 
			"""

	query = f""" 
		SELECT ce_dictionary.traditional, ce_dictionary.simplified, ce_dictionary.pinyin, ce_dictionary.english
		FROM ce_dictionary
		INNER JOIN hanzi_cedict ON ce_dictionary.index = hanzi_cedict.cedict_index
		WHERE hanzi_info.hanzi_index = {hanzi.hanzi_index};
		"""
	cursor.execute(query)
	rows = cursor.fetchall()

	if len(rows) == 0:
	    return None
	else:
	    return [Word(traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3]) for row in rows]

In [197]:
query_ce_dictionary_by_hanzi_index(conn,hanziTest)

OperationalError: near "index": syntax error

In [177]:
query_ce_dictionary_by_hanzi_index(conn,hanziTest)

AttributeError: 'sqlite3.Connection' object has no attribute 'fetchall'

In [168]:

def get_cedict_by_hanzi(conn: sqlite3.Connection, hanzi_list: List[Hanzi]) -> List[Word]:
    cursor = conn.cursor()
    #query = f"SELECT * FROM ce_dictionary WHERE simplified IN ({','.join(['?']*len(hanzi_list))}) OR traditional IN ({','.join(['?']*len(hanzi_list))})"
    query = f"""
    SELECT ce_dictionary.*
    FROM hanzi_cedict
    JOIN ce_dictionary ON hanzi_cedict.cedict_index = ce_dictionary.cedict_index
    WHERE hanzi_cedict.hanzi_index IN ({','.join(['?']*len(hanzi_list))})
    """
    hanzi_characters = [hanzi.hanzi for hanzi in hanzi_list]
    cursor.execute(query, hanzi_characters + hanzi_characters)
    rows = cursor.fetchall()
    if len(rows) == 0:
        return None
    else:OperationalError: ambiguous column name: ce_dictionary.traditional

        return [Word(traditional=row[0], simplified=row[1], pinyin=row[2], english=row[3], cedict_index=row[4]) for row in rows]


SyntaxError: invalid syntax (497087519.py, line 15)

In [139]:
for wrd in shared_words:
    print(wrd.simplified)

边
边
陲


In [140]:
get_table_schema('hanzi_cedict')

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'cedict_index', 'INTEGER', 0, None, 0),
 (2, 'hanzi_index', 'INTEGER', 0, None, 0)]

In [219]:
def get_cedict_by_hanzi(cursor, hanzi):
    
	query = f"""
	SELECT ce_dictionary.traditional, ce_dictionary.simplified, ce_dictionary.pinyin, ce_dictionary.english
	FROM ce_dictionary INNER JOIN hanzi_cedict
		ON ce_dictionary = hanzi_cedect.cedict_index
	WHERE hanzi_cedict.hanzi_index = {hanzi.hanzi_index};
	"""
	
	cursor.execute(query)

	rows = cursor.fetchall()

	return rows

In [220]:
get_cedict_by_hanzi(conn,hanziTest)

OperationalError: near "index": syntax error